In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
# import sage
# from scipy.stats import rankdata
from scipy.stats import ttest_ind, t
import sys
from helper import *
from kernelshap import *
# from os.path import join
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler
import warnings
warnings.filterwarnings('ignore')

/Users/jeremygoldwasser/Desktop/RankSHAP/kernelshap.py:285: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if var_method is "ls":
/Users/jeremygoldwasser/opt/anaconda3/envs/shap/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load dataset

In [30]:
np.random.seed(1)
data = pd.read_csv("Data/brca_small.csv")
X = data.values[:, :-1][:,:20]
Y = data.values[:, -1]
Y = (Y==2).astype(int) # Formulate as binary classification problem

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=100, random_state=0)

# Normalize
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std
d = X_train.shape[1]
mapping_dict = None

# Compute mean and covariance of training data
feature_means = np.mean(X_train, axis=0)
cov_mat = correct_cov(np.cov(X_train, rowvar=False))

In [31]:
# # df = sage.datasets.credit()
# # df.to_csv("Data/credit.csv")
# df = pd.read_csv("Data/credit.csv", index_col=0)

# # Property, other installment, housing, job, status of checking act, credit history, purpose, savings, employment since, marital status, old debtors
# n = df.shape[0]
# X_df = df.drop(["Good Customer"], axis=1)
# y = df["Good Customer"]

# categorical_columns = [
#     'Checking Status', 'Credit History', 'Purpose', #'Credit Amount', # It's listed but has 923 unique values
#     'Savings Account/Bonds', 'Employment Since', 'Personal Status',
#     'Debtors/Guarantors', 'Property Type', 'Other Installment Plans',
#     'Housing Ownership', 'Job', #'Telephone', 'Foreign Worker' # These are just binary
# ]
# X_binarized = pd.get_dummies(X_df, columns=categorical_columns)

# mapping_dict = {}
# for i, col in enumerate(X_df.columns):
#     bin_cols = []
#     for j, bin_col in enumerate(X_binarized.columns):
#         if bin_col.startswith(col):
#             bin_cols.append(j)
#     mapping_dict[i] = bin_cols

# np.random.seed(1)
# X_norm = (X_binarized-X_binarized.min())/(X_binarized.max()-X_binarized.min())
# n_train = round(n*0.8)
# train_idx = np.random.choice(n, n_train, replace=False)
# X_train, y_train = X_norm.iloc[train_idx].to_numpy(), y.iloc[train_idx].to_numpy()
# test_idx = np.setdiff1d(np.arange(n),train_idx)
# X_test, y_test = X_norm.iloc[test_idx].to_numpy(), y.iloc[test_idx].to_numpy()
# d = X_df.shape[1]
# d_bin = X_train.shape[1]

In [32]:
# Bank dataset
# import sage
# from sklearn.model_selection import train_test_split

# df = sage.datasets.bank()
# df.Default = df.Default.map({'no': 0, 'yes': 1})
# df.Housing = df.Housing.map({'no': 0, 'yes': 1})
# df.Loan = df.Loan.map({'no': 0, 'yes': 1})
# df.columns = df.columns.str.replace(' ', '_')
# df_bin = pd.get_dummies(df)

# # That's literally it. Note this mapping dict is a little different.
# train, test = train_test_split(
#     df_bin.values, test_size=int(0.1 * len(df_bin.values)), random_state=123)

# suc_idx = df_bin.columns.get_loc("Success")
# y_train = train[:, suc_idx].copy().astype(int)
# y_test = test[:, suc_idx].copy().astype(int)
# X_train = np.delete(train, suc_idx, axis=1).astype("float64")
# X_test = np.delete(test, suc_idx, axis=1).astype("float64")

# # Get mapping dict
# X_df = df_bin.drop(["Success"], axis=1)
# mapping_dict = {}
# for i, col in enumerate(df.columns):
#     bin_cols = []
#     for j, bin_col in enumerate(X_df.columns):
#         if bin_col.startswith(col):
#             bin_cols.append(j)
#     mapping_dict[i] = bin_cols

In [33]:
# # Adult census income dataset
# import shap
# X, y = shap.datasets.adult()
# X_display, y_display = shap.datasets.adult(display=True)
# X_binarized = pd.get_dummies(X_display)

# mapping_dict = {}
# for i, col in enumerate(X_display.columns):
#     bin_cols = []
#     for j, bin_col in enumerate(X_binarized.columns):
#         if bin_col.startswith(col):
#             bin_cols.append(j)
#     mapping_dict[i] = bin_cols

# X_norm = (X_binarized-X_binarized.min())/(X_binarized.max()-X_binarized.min())
# y_int = y_display.astype("int8")

# # Split into training and test sets
# np.random.seed(1)
# n, d = X_norm.shape
# n_train = round(n*0.75)
# train_idx = np.random.choice(n, size=n_train, replace=False)
# X_train_pd, y_train = X_norm.iloc[train_idx], y_int[train_idx]
# X_train = X_train_pd.to_numpy()

# test_idx = np.setdiff1d(list(range(n)), train_idx)
# X_test_pd, y_test = X_norm.iloc[test_idx], y_int[test_idx]
# X_test = X_test_pd.to_numpy()


### Define neural network, train, and move to numpy
- Trains with balanced weights

In [34]:
class NeuralNet(nn.Module):
    # def __init__(self, input_size, hidden_size, output_size):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(NeuralNet, self).__init__()
        # self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear1 = nn.Linear(input_size, hidden_size1)
        self.tanh = nn.Tanh()
        self.linear2 = nn.Linear(hidden_size1, hidden_size2)
        self.linear3 = nn.Linear(hidden_size2, output_size)
        # self.linear2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        out = self.linear1(x)
        out = self.tanh(out)
        out = self.linear2(out)
        out = self.tanh(out)
        out = self.linear3(out)
        out = self.softmax(out)
        return out

# Convert the input and label data to PyTorch tensors
inputs = torch.tensor(X_train, dtype=torch.float32)
labels = torch.tensor(y_train, dtype=torch.long)

# Compute the class weights
class_counts = torch.bincount(labels)
num_samples = len(labels)
class_weights = 1.0 / class_counts.float()
sample_weights = class_weights[labels]

# Create a sampler with balanced weights
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=num_samples, replacement=True)

# Create a DataLoader with the sampler
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset, batch_size=32, sampler=sampler)
# dataloader = DataLoader(dataset, batch_size=32)

torch.manual_seed(0)

# Create an instance
# net = NeuralNet(input_size=X_train.shape[1], hidden_size=50, output_size=2)
net = NeuralNet(input_size=X_train.shape[1], hidden_size1=100, hidden_size2=50, output_size=2)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)#.01

# Iterate over the training data in batches
num_epochs = 20

for epoch in range(num_epochs):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    # print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

def neural_net(x):
    output = net(x)[0,1] if x.shape[0]==1 else net(x)[:,1]
    return output

def compute_hessian(x):
    if not torch.is_tensor(x):
        x = torch.tensor(x, dtype=torch.float32)
    dim = x.shape[1]
    hessian = torch.autograd.functional.hessian(neural_net, x)
    hessian = hessian.reshape((dim,dim)).detach().numpy()
    return hessian

def fmodel(x):
    if not torch.is_tensor(x):
        x = torch.tensor(x, dtype=torch.float32)
    return neural_net(x).detach().numpy()

print("Class imbalance: {}%".format(round(100*(max(np.mean(y_test), 1-np.mean(y_test))))))
Y_preds = (fmodel(X_test) > 0.5).astype("int")
print("NN {}% accuracy".format(round(np.mean(Y_preds == y_test)*100)))



Class imbalance: 55%
NN 79% accuracy


In [35]:
%run helper
# Compute mean and covariance of training data
feature_means = np.mean(X_train, axis=0)
cov_mat = correct_cov(np.cov(X_train, rowvar=False))

# Select point and compute its gradient and hessian
xloc = X_test[0:1]
xloc_torch = torch.tensor(xloc, dtype=torch.float32).requires_grad_(True)
y_pred = net(xloc_torch)[0,1]
y_pred.backward()
gradient = xloc_torch.grad.detach().numpy().reshape((xloc.shape[1], 1))
hessian = compute_hessian(xloc)

# Obtain true SHAP values and verify their feasibility
true_shap_vals = compute_shap_vals_quadratic(xloc, gradient, hessian, feature_means, cov_mat, mapping_dict=mapping_dict)

y_pred = y_pred.detach().numpy()
def approx(input):
    return f_second_order_approx(y_pred,input,xloc,gradient,hessian)

print(approx(xloc) - np.mean(approx(X_train)))
print(np.sum(true_shap_vals))

-8.97556459367533e-05
-8.9805e-05


## KernelSHAP
Doesn't converge w/ LS variance bc it's too high --> test stats too low

- We still do the new t-test here

In [21]:
%run kernelshap
# np.random.seed(1)
K = 3
kshap_ests, kshap_vars, n_subsets = kernelshap(approx, X_train, xloc, K, 
            n_perms_btwn_tests=500, n_samples_per_perm=10, alpha=0.2, n_init=500, #n=50,
            mapping_dict=mapping_dict, max_n_perms=10000, var_method="ls")
print(n_subsets)
print(get_ordering(true_shap_vals))
print(get_ordering(kshap_ests))
print(np.sum(kshap_ests))

KeyboardInterrupt: 

Bootstrapped variance works!

In [75]:
%run kernelshap
# np.random.seed(1)
K = 3
kshap_ests, kshap_vars, n_subsets = kernelshap(approx, X_train, xloc, K, 
            n_perms_btwn_tests=500, n_samples_per_perm=10, alpha=0.2, n_init=500, #n=50,
            mapping_dict=mapping_dict, max_n_perms=50000, var_method="boot")
print(n_subsets)
print(get_ordering(true_shap_vals))
print(get_ordering(kshap_ests))
print(np.sum(kshap_ests))

23000
[ 8  3  2  4  7  0 10 11  5  9  6  1]
[ 8  3  2  4  7  0 10 11  5  6  1  9]
0.025785144797551167


Unbiased method does not produce SHAP values that converge. Maybe there's a bug... not sure what's going on.

In [40]:
kshap_ests, kshap_vars, n_subsets = kernelshap(approx, X_train, xloc, K, 
            n_perms_btwn_tests=5000, n_samples_per_perm=10, alpha=0.2, n_init=500, #n=50,
            mapping_dict=mapping_dict, max_n_perms=50000, unbiased=True)
print(get_ordering(kshap_ests))
print(np.sum(kshap_ests))

Reached max number of permutations.
[ 3  8  5  0  9  2  6  7  4 10 11  1]
0.025785144797551188


## Simulation

K=1: Control FWER! 20% w/ 50 sims, alpha=0.2
- This is with bootstrapped variance and standard KernelSHAP. For n in t-test DF, just use # perms.
- LS variance does not work, nor does unbiased KernelSHAP.

K=3: Often reaches 10k perms and cuts out... but that's OK: The FWER is 40%, indicating it hasn't converged.

In [31]:
%run kernelshap
np.random.seed(1)
K = 1
true_order = get_ordering(true_shap_vals)
true_top_K = true_order[:K]
print(true_top_K)
print("#"*20)
same_top_K = []
for i in range(50):
    print(i)
    kshap_ests, kshap_vars, n_subsets = kernelshap(approx, X_train, xloc, K, 
            n_perms_btwn_tests=500, n_samples_per_perm=10, alpha=0.2, n_init=500, #n=50,
            mapping_dict=mapping_dict, max_n_perms=10000, unbiased=True)#var_method="ls")
    est_top_K = np.argsort(np.abs(kshap_ests))[::-1][:K]

    has_same_top_K = np.array_equal(true_top_K, est_top_K)
    same_top_K.append(has_same_top_K)
    if (i+1) % 5==0: print(est_top_K, n_subsets, 1-np.mean(same_top_K), i+1)
    
1-np.mean(same_top_K)

[8]
####################
0
Reached max number of permutations.
1
Reached max number of permutations.
2
Reached max number of permutations.
3
Reached max number of permutations.
4


KeyboardInterrupt: 

### Unbiased KernelSHAP doesn't work
I don't think there's a bug?
- Running for 50k perms: Usually gets top 5 right, but they're out of order.
- Generally, yeah, much more variable.

In [68]:
%run kernelshap
true_top_K = get_ordering(true_shap_vals)[:K]
print(true_top_K)

kshap_ests, kshap_vars, n_subsets = kernelshap(approx, X_train, xloc, K,
        n_perms_btwn_tests=500, n_samples_per_perm=20,
        mapping_dict=mapping_dict, alpha=0.2, n_init=500, unbiased=True,
        perm_test=False, max_n_perms=50000)
est_top_K = get_ordering(kshap_ests)[:K]
print(est_top_K)

print(get_ordering(kshap_ests))
print(get_ordering(true_shap_vals))
print(n_subsets)

[8 3 2]
Reached max number of permutations.
[5 3 8]
[ 5  3  8  2  7  4  9  1  0 10  6 11]
[ 8  3  2  4  7  0 10 11  5  9  6  1]
50000


Weirdly, the permutation test says it converges when it obviously hasn't.

In [69]:
kshap_ests, kshap_vars, n_subsets = kernelshap(approx, X_train, xloc, K,
        n_perms_btwn_tests=500, n_samples_per_perm=20,
        mapping_dict=mapping_dict, alpha=0.2, n_init=5000, unbiased=True,
        perm_test=True, max_n_perms=50000)
est_top_K = get_ordering(kshap_ests)[:K]
print(true_top_K)
print(est_top_K)

print(get_ordering(kshap_ests))
print(get_ordering(true_shap_vals))
print(n_subsets)

[8 3 2]
[6 5 0]
[ 6  5  0  3  4  8  7  2  9  1 10 11]
[ 8  3  2  4  7  0 10 11  5  9  6  1]
5000


## Run many times on NN 

K=1: 20 perms btwn tests: FWER 16%
- Higher number --> FWER goes to 0

K=2: 20 perms btwn tests: FWER 16%
K=3: gets stuck/ too slow

In [38]:
# np.random.seed(2)
K = 1
top_K2_list = []
max_n_perms = 2000
# for i in range(50):
ct = 0
while ct < 50:
    # print(i)
    kshap_ests, kshap_vars, n_subsets = kernelshap(fmodel, X_train, xloc, K,
            n_perms_btwn_tests=20, n_samples_per_perm=20,
            mapping_dict=mapping_dict,  var_method="boot",#n=20,
            alpha=0.2, n_init=200, max_n_perms=max_n_perms)
    if n_subsets < max_n_perms:
        # print("here")
        est_top_K = get_ordering(kshap_ests)[:K]
        top_K2_list.append(est_top_K)
        ct += 1
        if ct % 5==0: #print(ct)
            top_K2 = np.array(top_K2_list)
            most_common_row = mode_rows(top_K2)
            ct2 = 0
            for i in range(top_K2.shape[0]):
                if np.array_equal(most_common_row, top_K2[i]):
                    ct2 += 1
            print(ct, np.round(1 - ct2 / top_K2.shape[0], 2)) # FWER

here
Reached max number of permutations.
here
here
here
here
5 0.4
Reached max number of permutations.
Reached max number of permutations.
here
here


KeyboardInterrupt: 